# Experimental Agent Creation Functions 1

In [14]:
#Imports
import os
import gym
import numpy as np
import keras
from keras.models import Model, load_model
from keras.layers import Input, Dense, Lambda, Add
from tensorflow.keras.optimizers import Adam
from keras import backend as kb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
import torch
import gym
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
from network import MlpPolicy

In [13]:
EPISODES = 1000

class PPOAgent:
    def __init__(self, env_name):
        self.model_name = "PPO"
        self.env = gym.make(env_name)
        self.env_name = "NASim_RL_Model"
        self.action_size = self.env.action_space.n
        self.state_size = 10
        self.gamma = 0.97
        self.learning_rate = 0.001
        self.states = []
        self.gradients = []
        self.rewards = []
        self.probabilities = []
        self.model = self.build_model()
        self.model.summary()
        
    def build_model(self):
        model = Sequential().build(self.state_size)
        opt = Adam(lr=self.learning_rate)
        model.compile(loss = tf.losses.Huber(), optimizer = opt)
        return model
    
    def act(self, state):
        action_prob = self.model.predict(state, batch_size = 1).flatten()
        self.probabilities.append(action_prob)
        prob = action_prob / np.sum(action_prob)
        action = np.random.choice(self.action_size, 1, p = prob)[0]
        return action, prob
    
    def remember(self, state, action, prob, reward):
        rem_arr = np.zeroes([self.action_size])
        rem_arr[action] = 1
        self.gradients.append(np.array(y).astype('float32') - prob)
        self.states.append(state)
        self.rewards.append(reward)
        
    def train(self):
        gradients = np.vstack(self.gradients)
        rewards = np.vstack(self.rewards)
        rewards = self.discount_rewards(rewards)
        rewards = rewards / np.std(rewards - np.mean(rewards))
        gradients *= rewards
        X = np.squeeze(np.vstack([self.states]))
        Y = self.probs + self.learning_rate * np.squeeze(np.vstack([gradients]))
        self.model.train_on_batch(X, Y)
        self.states, self.probs, self.gradients, self.rewards = [], [], [], []
    
    def load(self, name):
        self.model.load_weights(name)
    
    def save(self, name):
        self.model.save_weights(name)
        
    def run(self):
        state = env.reset()
        score = 0
        episode = 0
        agent = PPOAgent(env.action_space)
    
        while True:
            env.render()
            action, prob = agent.act(state)
            state, reward, done, info = env.step(action)
            score += reward
            agent.remeber(state.ravel(), action, prob, reward)
        
            if done:
                episode += 1
                agent.train()
                print('Episode: %d - Score: %f.' % (episode, score))
                score = 0
                state = env.reset()
                prev_x = None
                if episode > 1 and episode % 10 == 0:
                    agent.save('./save_model/NASim_RL_Model.h5')
    

In [15]:
if __name__ == "__main__":
    agent = PPOAgent("nasim:Tiny-v0")

        

TypeError: 'int' object is not iterable